In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## Template matching
Objetivos:
1. Obtener una detección del logo en cada imagen sin falsos positivos
2. Plantear y validar un algoritmo para múltiples detecciones en la imagen `coca_multi.png` con el mismo template del ítem 1
3. Generalizar el algoritmo del ítem 2 para todas las imágenes

Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección.

### Template e imagenes a analizar

In [ ]:
TEMPLATE_PATH = "./template/pattern.png"
IMAGES_PATH = "./images/"

In [ ]:
template = cv2.imread(TEMPLATE_PATH, cv2.IMREAD_COLOR)
plt.imshow(cv2.cvtColor(template, cv2.COLOR_BGR2RGB))

In [ ]:
images = [
    cv2.imread(IMAGES_PATH + image_name, cv2.IMREAD_COLOR)
    for image_name in os.listdir(path=IMAGES_PATH)
]

fig, axs = plt.subplots(nrows=1, ncols=len(images))
fig.set_size_inches(25, 5)
for ax, image in zip(axs, images):
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

### Aplicación de pirámides en el template

In [ ]:
# Get max width and height of the images to test
max_height = max([image.shape[0] for image in images])
min_height = min([image.shape[0] for image in images])
max_width = max([image.shape[1] for image in images])
min_width = min([image.shape[1] for image in images])

# over-sampling
template_oversapling = [template.copy()]
while (template_oversapling[-1].shape[0] < max_height) and (
    template_oversapling[-1].shape[1] < max_width
):
    template_oversapling.append(cv2.pyrUp(template_oversapling[-1]))

# sub-sampling
template_subsapling = [template.copy()]
while (template_subsapling[-1].shape[0] > min_height) and (
    template_subsapling[-1].shape[1] > min_width
):
    template_subsapling.append(cv2.pyrDown(template_subsapling[-1]))

# to not repeat the original
template_pyramid = template_subsapling + template_oversapling[1:] 

In [ ]:
print("NUMBER OF TEMPLATES:", len(template_pyramid))

fig, axs = plt.subplots(nrows=1, ncols=len(template_pyramid))
fig.set_size_inches(20, 5)
for ax, template in zip(axs, template_pyramid):
    ax.imshow(cv2.cvtColor(template, cv2.COLOR_BGR2RGB))

### Exploración de métricas

In [ ]:

metrics = [
    ("TM_CCOEFF", cv2.TM_CCOEFF),
    ("TM_CCOEFF_NORMED", cv2.TM_CCOEFF_NORMED),
    ("TM_CCORR", cv2.TM_CCORR),
    ("TM_CCORR_NORMED", cv2.TM_CCORR_NORMED),
    ("TM_SQDIFF", cv2.TM_SQDIFF),
    ("TM_SQDIFF_NORMED", cv2.TM_SQDIFF_NORMED),
]

fig, axs = plt.subplots(nrows=1, ncols=len(metrics))

for ax, (metric_name, metric_fn) in zip(axs, metrics):
    image = images[0].copy()
    match = cv2.matchTemplate(image, template, metric_fn)
    min_value, max_value, min_location, max_location = cv2.minMaxLoc(match)
    if metric_name in ["TM_SQDIFF", "TM_SQDIFF_NORMED"]:
        top_left = min_location
    else:
        top_left = max_location
    bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])

    cv2.rectangle(image, top_left, bottom_right, 255, 2)
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


### Resources
- [Image Processing in OpenCV: Template matching](https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html)
- [Image Processing: Image Pyramids](https://docs.opencv.org/4.x/d4/d1f/tutorial_pyramids.html)